In [20]:
import tensorflow as tf
from ResNet import resnet_graph
import tensorflow_datasets as tfds
from datetime import datetime
from tensorflow.keras.layers import *

In [ ]:
tfds.disable_progress_bar()

In [9]:
# ds, dsi = tfds.load('cats_vs_dogs',data_dir='dataset', split='train', with_info=True)
# train_raw   = tf.data.TFRecordDataset(['dataset/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord-00000-of-00008',
#                                        'dataset/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord-00001-of-00008',
#                                        'dataset/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord-00002-of-00008',
#                                        'dataset/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord-00003-of-00008'])
# test_raw    = tf.data.TFRecordDataset(['dataset/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord-00004-of-00008',
#                                        'dataset/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord-00005-of-00008',
#                                        'dataset/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord-00006-of-00008',
#                                        'dataset/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord-00007-of-00008'])

In [33]:
ds = tfds.load('mnist')

In [13]:
# def parse(example):
#     LABELED_ID_TFREC_FORMAT = {
#         "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
#         "label": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
#     }
#     example = tf.io.parse_single_example(example, LABELED_ID_TFREC_FORMAT)
#     image = tf.image.decode_jpeg(example['image'], channels=3)
#     label = tf.cast(example['label'], tf.int32)
#     image = tf.image.resize(image,(256,256))/255
#     return image, tf.cond(
#         tf.equal(label,1),
#         lambda : tf.constant([0.0,1.0]),
#         lambda : tf.constant([1.0,0.0]))
#
# train = train_raw.shuffle(1000).take(32).map(parse).batch(4).prefetch(2)
# test  =  test_raw.shuffle(1000).take(8).map(parse).batch(4).prefetch(2)

In [42]:
input_image = Input((256, 256, 1))
x = resnet_graph(input_image, False)
x= GlobalMaxPooling2D()(x[-1])
x = Dense(2, activation=tf.nn.softmax)(x)
model = tf.keras.Model(input_image,x)

In [43]:
def f(i):
    im = tf.image.resize(i['image'],(256,256))
    return im, i['label']

In [44]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
logdir="logs"
tensorboard_callback = tf.keras.callbacks.TensorBoard(write_graph=True, log_dir=logdir)
model.fit(ds['train'].map(f).take(100).batch(10),
          epochs=10,
          callbacks=[tensorboard_callback])

Epoch 1/10
      4/Unknown - 331s 83s/step - loss: 442852.0738 - categorical_accuracy: 0.3333

KeyboardInterrupt: 

In [24]:
# for im, l in ds['train'].map(f).take(1):
#     print(l)



